In [17]:
from flask_restful import Api,Resource
from flask_restful import reqparse
from flask import jsonify
import pymysql
import json
import flask
import csv

DB_HOST = "localhost"
DB_USER = "team1"
DB_PASSWORD = "123456"
DB_SCHEMA = "testdb"

parser = reqparse.RequestParser()
# parser.add_argument("name")
# parser.add_argument("gender")
# parser.add_argument("birth")
# parser.add_argument("note")

response = {"code": 200, "msg": "success"}


class User(Resource):
    def db_init(self):
        db = pymysql.connect(DB_HOST, DB_USER, DB_PASSWORD, DB_SCHEMA)
        cursor = db.cursor(pymysql.cursors.DictCursor)
        return db, cursor

    def get(self, id):
        db, cursor = self.db_init()
        sql = """Select * FROM testdb.course Where id = {}""".format(id)
        cursor.execute(sql)
        db.commit()
        user = cursor.fetchall()
        db.close()
        response["data"] = user
        return response

    def put(self, id):
        db, cursor = self.db_init()
        arg = parser.parse_args()
        user = {
            "name": arg["name"],
            "gender": arg["gender"],
#             "birth": arg["birth"]
        }
        query = []
        for key, value in user.items():
            print(key)
            print(value)
            if value != None:
                query.append(key + " = " + " '{}' ".format(value))
        query = ",".join(query)

        sql = """Update testdb.course Set {} Where id = {}""".format(query, id)
        result = cursor.execute(sql)
        db.commit()
        db.close()

        response["result"] = True if result == 1 else False

        return response

    def delete(self, id):
        db, cursor = self.db_init()
        sql = """Delete FROM testdb.users Where id = {}""".format(id)
        result = cursor.execute(sql)
        db.commit()
        db.close()

        response["result"] = True if result == 1 else False
        return response


class Users(Resource):
    def db_init(self):
        db = pymysql.connect(DB_HOST, DB_USER, DB_PASSWORD, DB_SCHEMA)
        cursor = db.cursor(pymysql.cursors.DictCursor)
        return db, cursor

    def get(self):
        db, cursor = self.db_init()
        sql = """Select * FROM testdb.course """
        cursor.execute(sql)
        db.commit()
        users = cursor.fetchall()
        db.close()
        return response

    def post(self):
        db, cursor = self.db_init()
        excelDataArry = []
        with open('D:/0920_Hackathon_PRD/pccu_class.csv', newline='') as csvfile:
            rows = csv.reader(csvfile)
            for row in rows:
                user = {"name": row[0],"total_time": row[1],"price": row[2],"sale_price":row[3],
                        "early_price":row[4],"start_time":row[5],"status":row[6],"batch":row[7],
                        "location":row[8],"teacher":row[9],"cooperation":row[10],"category":row[11],"type":row[12],"source":row[13]}
                sql = """Insert into testdb.course_pcc (name, total_time, price,sale_price,
                early_price,start_time,status,batch,location,teacher,cooperation,category,type,source) values('{}', '{}', '{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}')""".format(user["name"], user["total_time"],user["price"],user["sale_price"],user["early_price"],user["start_time"],user["status"],user["batch"],user["location"],user["teacher"],user["cooperation"],user["category"],user["type"],user["source"])
                result = cursor.execute(sql)
                db.commit()
        db.close()
        response["result"] = True if result == 1 else False

        return response




app = flask.Flask(__name__)
app.config["DEBUG"] = True

api = Api(app)

api.add_resource(User, "/user/<id>")
api.add_resource(Users, "/users")

@app.route("/", methods=["GET"])
def home():
    return "Hello World"

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=80, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Sep/2019 16:16:05] "POST /users HTTP/1.1" 200 -
